In [ ]:
{
  "cells": [
    {
      "cell_type": "code",
      "execution_count": 1,
      "metadata": {
        "id": "I84fo6i_gNux"
      },
      "outputs": [],
      "source": [
        "# Step 0: Import necessary libraries\n",
        "\n",
        "import torch  # PyTorch - core DL framework\n",
        "\n",
        "from datasets import load_dataset  # For loading datasets from Hugging Face\n",
        "\n",
        "from transformers import (\n",
        "    AutoTokenizer, AutoModelForSeq2SeqLM,  # Tokenizer & model loader\n",
        "    Trainer, TrainingArguments, DataCollatorForSeq2Seq  # For training setup\n",
        ")\n",
        "\n",
        "import evaluate  # For evaluation metrics\n",
        "\n",
        "import psutil  # To check system memory\n",
        "\n",
        "import time  # To measure training time\n"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 2,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "TaaAKmNqLB6N",
        "outputId": "b60cbcaa-8596-4ff6-9e54-b1e56bcd8b6f"
      },
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Total RAM: 12.67 GB\n",
            "Available RAM: 10.72 GB\n"
          ]
        }
      ],
      "source": [
        "# Step 1: Display system memory info\n",
        "print(f\"Total RAM: {psutil.virtual_memory().total / (1024 ** 3):.2f} GB\")\n",
        "print(f\"Available RAM: {psutil.virtual_memory().available / (1024 ** 3):.2f} GB\")"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 3,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "GoDfsVecMIQJ",
        "outputId": "7be5355a-41b4-47bd-f19a-4bf52b2d5916"
      },
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "\n",
            "Loading the Bitext customer support dataset...\n"
          ]
        },
        {
          "output_type": "stream",
          "name": "stderr",
          "text": [
            "/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: \n",
            "The secret `HF_TOKEN` does not exist in your Colab secrets.\n",
            "To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.\n",
            "You will be able to reuse this secret in all of your notebooks.\n",
            "Please note that authentication is recommended but still optional to access public models or datasets.\n",
            "  warnings.warn(\n"
          ]
        },
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Loaded 26872 samples.\n"
          ]
        }
      ],
      "source": [
        "# Step 2: Load the Bitext customer support dataset\n",
        "print(\"\\nLoading the Bitext customer support dataset...\")\n",
        "dataset = load_dataset(\"bitext/Bitext-customer-support-llm-chatbot-training-dataset\")\n",
        "\n",
        "full_data = dataset['train']\n",
        "print(f\"Loaded {len(full_data)} samples.\")\n"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 4,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "wdp0CY2zMIVd",
        "outputId": "920fab45-5564-46c1-c57d-b412c7cc6342"
      },
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Split into 21497 training and 5375 validation samples.\n"
          ]
        }
      ],
      "source": [
        "# Step 3: Split the dataset into training and evaluation (80:20)\n",
        "\n",
        "# We'll use 80% of the data for training and keep 20% aside for validation\n",
        "split_dataset = full_data.train_test_split(test_size=0.2, seed=42)\n",
        "\n",
        "# Extract the training and validation sets\n",
        "train_data = split_dataset[\"train\"]\n",
        "eval_data = split_dataset[\"test\"]\n",
        "\n",
        "# Quick check on how many samples we got in each set\n",
        "print(f\"Split into {len(train_data)} training and {len(eval_data)} validation samples.\")\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "u5GqcIHybuD3"
      },
      "source": []
    },
    {
      "cell_type": "code",
      "execution_count": 5,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "Izx9HGpHMIa5",
        "outputId": "85b72b95-b20d-4e01-d319-edd51a04a869"
      },
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "\n",
            "Loading FLAN-T5 model and tokenizer...\n"
          ]
        }
      ],
      "source": [
        "# Step 4: Load FLAN-T5 model and tokenizer\n",
        "print(\"\\nLoading FLAN-T5 model and tokenizer...\")\n",
        "\n",
        "# Specify the model checkpoint to use\n",
        "model_name = \"google/flan-t5-base\"\n",
        "\n",
        "# Load the tokenizer for processing text inputs\n",
        "tokenizer = AutoTokenizer.from_pretrained(model_name)\n",
        "\n",
        "# Load the pre-trained FLAN-T5 model for sequence-to-sequence tasks\n",
        "model = AutoModelForSeq2SeqLM.from_pretrained(model_name)\n"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 6,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 98,
          "referenced_widgets": [
            "c777f1d7b051452dacd3a15722064ac7",
            "aa6463e797284971b88ac46c38dc46c1",
            "ac4949791ae3483c87960b9af72164b4",
            "ca351d8c794740df8a627b8774f98f6d",
            "8ab3b9b7030d4882ab5c8563e9a8951b",
            "42973b38546c421aae760a0db8a848d5",
            "6c79457fd1ee4e20b83b5d7058d17a37",
            "d4acf8efe5834f9abed1b63bd983c97a",
            "01ecbe20bf6648588cbddb378632a7b7",
            "102c219e9e0044618895f0cc27a01a80",
            "00e597996aac4637a8cbd3057c04d19a",
            "0b5fa0aa83bb43d79b424c10464d8151",
            "843762fb47b84b408106c9e3a55a156c",
            "5095d04062af413cb4fbc4d66009056e",
            "c0359cf4432f4b5e9b5eef4d7aab21ea",
            "70b7a6fd920a4e50bca9b29fb0f0dc9c",
            "3f9959767bd8400fac30d7a53e787a67",
            "403a3fdb37e24846a503a992e5eeea54",
            "4dd33e1517684c92a1da2fb22a2a5368",
            "b78dfd88cb01423abe75df88048f77c4",
            "670e28c91f8f49b58acf64ad10a9db51",
            "561b83b423cc438f86ba3ca5f17c3aaf"
          ]
        },
        "id": "CTapjZ4lMIdr",
        "outputId": "b7ddc4d5-567f-49e5-f088-baee114b60a0"
      },
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "🔄 Tokenizing the dataset...\n"
          ]
        },
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "Map:   0%|          | 0/21497 [00:00<?, ? examples/s]"
            ],
            "application/vnd.jupyter.widget-view+json": {
              "version_major": 2,
              "version_minor": 0,
              "model_id": "c777f1d7b051452dacd3a15722064ac7"
            }
          },
          "metadata": {}
        },
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "Map:   0%|          | 0/5375 [00:00<?, ? examples/s]"
            ],
            "application/vnd.jupyter.widget-view+json": {
              "version_major": 2,
              "version_minor": 0,
              "model_id": "0b5fa0aa83bb43d79b424c10464d8151"
            }
          },
          "metadata": {}
        }
      ],
      "source": [
        "# Step 5: Tokenize inputs and targets to get model-ready data\n",
        "def preprocess(example):\n",
        "    # Combine instruction with a clear prompt so the model understands the task\n",
        "    input_text = \"Answer the customer query: \" + example[\"instruction\"]\n",
        "    target_text = example[\"response\"]\n",
        "\n",
        "    # Tokenize the input text, making sure it's the same length for batching\n",
        "    inputs = tokenizer(input_text, max_length=128, padding=\"max_length\", truncation=True)\n",
        "    # Tokenize the target/response text similarly\n",
        "    targets = tokenizer(target_text, max_length=128, padding=\"max_length\", truncation=True)\n",
        "\n",
        "    # Assign the target tokens as labels that the model should predict\n",
        "    inputs[\"labels\"] = targets[\"input_ids\"]\n",
        "    return inputs\n",
        "\n",
        "# Apply the preprocessing function to both training and validation datasets\n",
        "# Removing original columns since we only need tokenized data for training\n",
        "print(\"🔄 Tokenizing the dataset...\")\n",
        "tokenized_train = train_data.map(preprocess, remove_columns=train_data.column_names)\n",
        "tokenized_eval = eval_data.map(preprocess, remove_columns=eval_data.column_names)\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "altjMFyyL6fK"
      },
      "source": []
    },
    {
      "cell_type": "code",
      "execution_count": 7,
      "metadata": {
        "id": "6DrSFTvhN0yg"
      },
      "outputs": [],
      "source": [
        "# Step 6: Set up training parameters for the Trainer\n",
        "training_args = TrainingArguments(\n",
        "    output_dir=\"voxa_model_cpu\",               # Where to save model checkpoints and final model\n",
        "    eval_steps=500,                            # Check performance every 500 steps (not just epochs)\n",
        "    save_strategy=\"epoch\",                     # Save model at the end of each epoch\n",
        "    learning_rate=2e-4,                        # How fast the model learns (tweak if needed)\n",
        "    per_device_train_batch_size=2,             # Keep batch size small to avoid running out of memory\n",
        "    num_train_epochs=2,                        # Go through the whole dataset once\n",
        "    weight_decay=0.01,                         # Small regularization to prevent overfitting\n",
        "    logging_dir=\"logs\",                        # Folder to store training logs\n",
        "    logging_steps=20,                          # Log training info every 20 steps\n",
        "    save_total_limit=2,                        # Keep only 2 recent saved models to save space\n",
        "    report_to=\"none\",                          # Turn off external logging services like WandB\n",
        "    fp16=False,                                # Use full precision since CPU doesn't support half precision\n",
        "    dataloader_num_workers=0,                  # Use single thread for loading data, safer on limited RAM\n",
        "    gradient_accumulation_steps=4              # No gradient accumulation here, batch size stays 1\n",
        ")\n"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 8,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "82XeQJ2DN06a",
        "outputId": "8a4826cd-13b5-4748-e748-2b9ad4d9e60d"
      },
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "\n",
            "Setting up Trainer...\n"
          ]
        },
        {
          "output_type": "stream",
          "name": "stderr",
          "text": [
            "<ipython-input-8-c9f6bc017adc>:7: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.\n",
            "  trainer = Trainer(\n"
          ]
        }
      ],
      "source": [
        "# step 7: Data collator automatically pads and batches sequences for us, so no manual padding needed\n",
        "print(\"\\nSetting up Trainer...\")\n",
        "\n",
        "data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)\n",
        "\n",
        "# Trainer class wraps everything — it handles the training loop, evaluation, saving checkpoints, and more\n",
        "trainer = Trainer(\n",
        "    model=model,                      # Our FLAN-T5 model\n",
        "    args=training_args,               # All the training configs we set earlier\n",
        "    train_dataset=tokenized_train,   # Tokenized training data\n",
        "    eval_dataset=tokenized_eval,     # Tokenized validation data\n",
        "    tokenizer=tokenizer,              # So trainer knows how to decode tokens during eval\n",
        "    data_collator=data_collator      # Helps prepare batches dynamically\n",
        ")\n"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 9,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "ksTeXOVdN08U",
        "outputId": "c4cbfd50-7060-45c8-86bf-b310b0a58807"
      },
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Memory usage before training: 10.41 GB\n"
          ]
        }
      ],
      "source": [
        "# Step 8: Check memory before training\n",
        "print(f\"Memory usage before training: {psutil.virtual_memory().available / (1024 ** 3):.2f} GB\")\n"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 10,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 1000
        },
        "id": "xgixfsDDOhYE",
        "outputId": "3dcaceef-db22-4618-c515-1e62d1d9d958"
      },
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "\n",
            "Starting training...\n"
          ]
        },
        {
          "output_type": "stream",
          "name": "stderr",
          "text": [
            "Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.\n"
          ]
        },
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<IPython.core.display.HTML object>"
            ],
            "text/html": [
              "\n",
              "    <div>\n",
              "      \n",
              "      <progress value='5374' max='5374' style='width:300px; height:20px; vertical-align: middle;'></progress>\n",
              "      [5374/5374 1:03:05, Epoch 1/2]\n",
              "    </div>\n",
              "    <table border=\"1\" class=\"dataframe\">\n",
              "  <thead>\n",
              " <tr style=\"text-align: left;\">\n",
              "      <th>Step</th>\n",
              "      <th>Training Loss</th>\n",
              "    </tr>\n",
              "  </thead>\n",
              "  <tbody>\n",
              "    <tr>\n",
              "      <td>20</td>\n",
              "      <td>5.356300</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>40</td>\n",
              "      <td>2.117000</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>60</td>\n",
              "      <td>1.546700</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>80</td>\n",
              "      <td>1.388500</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>100</td>\n",
              "      <td>1.220500</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>120</td>\n",
              "      <td>1.217900</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>140</td>\n",
              "      <td>1.090900</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>160</td>\n",
              "      <td>1.130100</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>180</td>\n",
              "      <td>1.039900</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>200</td>\n",
              "      <td>1.053500</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>220</td>\n",
              "      <td>1.070800</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>240</td>\n",
              "      <td>0.986000</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>260</td>\n",
              "      <td>1.047500</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>280</td>\n",
              "      <td>0.975300</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>300</td>\n",
              "      <td>0.999000</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>320</td>\n",
              "      <td>0.974000</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>340</td>\n",
              "      <td>0.949900</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>360</td>\n",
              "      <td>0.944200</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>380</td>\n",
              "      <td>0.919900</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>400</td>\n",
              "      <td>0.959300</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>420</td>\n",
              "      <td>0.914000</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>440</td>\n",
              "      <td>0.904100</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>460</td>\n",
              "      <td>0.898500</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>480</td>\n",
              "      <td>0.896100</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>500</td>\n",
              "      <td>0.859300</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>520</td>\n",
              "      <td>0.918000</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>540</td>\n",
              "      <td>0.877100</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>560</td>\n",
              "      <td>0.885300</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>580</td>\n",
              "      <td>0.818200</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>600</td>\n",
              "      <td>0.844100</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>620</td>\n",
              "      <td>0.841500</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>640</td>\n",
              "      <td>0.866200</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>660</td>\n",
              "      <td>0.849100</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>680</td>\n",
              "      <td>0.888500</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>700</td>\n",
              "      <td>0.830000</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>720</td>\n",
              "      <td>0.807100</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>740</td>\n",
              "      <td>0.842500</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>760</td>\n",
              "      <td>0.852300</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>780</td>\n",
              "      <td>0.821000</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>800</td>\n",
              "      <td>0.818200</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>820</td>\n",
              "      <td>0.833200</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>840</td>\n",
              "      <td>0.828400</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>860</td>\n",
              "      <td>0.763400</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>880</td>\n",
              "      <td>0.836400</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>900</td>\n",
              "      <td>0.826700</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>920</td>\n",
              "      <td>0.835300</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>940</td>\n",
              "      <td>0.810100</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>960</td>\n",
              "      <td>0.778400</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>980</td>\n",
              "      <td>0.785300</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1000</td>\n",
              "      <td>0.768200</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1020</td>\n",
              "      <td>0.793100</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1040</td>\n",
              "      <td>0.794100</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1060</td>\n",
              "      <td>0.760300</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1080</td>\n",
              "      <td>0.772800</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1100</td>\n",
              "      <td>0.790000</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1120</td>\n",
              "      <td>0.790500</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1140</td>\n",
              "      <td>0.829100</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1160</td>\n",
              "      <td>0.773200</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1180</td>\n",
              "      <td>0.784200</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1200</td>\n",
              "      <td>0.803200</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1220</td>\n",
              "      <td>0.771600</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1240</td>\n",
              "      <td>0.786100</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1260</td>\n",
              "      <td>0.761800</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1280</td>\n",
              "      <td>0.730300</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1300</td>\n",
              "      <td>0.738600</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1320</td>\n",
              "      <td>0.725900</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1340</td>\n",
              "      <td>0.784900</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1360</td>\n",
              "      <td>0.716100</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1380</td>\n",
              "      <td>0.749900</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1400</td>\n",
              "      <td>0.745600</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1420</td>\n",
              "      <td>0.725800</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1440</td>\n",
              "      <td>0.701800</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1460</td>\n",
              "      <td>0.739700</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1480</td>\n",
              "      <td>0.732100</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1500</td>\n",
              "      <td>0.772200</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1520</td>\n",
              "      <td>0.717900</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1540</td>\n",
              "      <td>0.721700</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1560</td>\n",
              "      <td>0.747500</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1580</td>\n",
              "      <td>0.747600</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1600</td>\n",
              "      <td>0.729500</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1620</td>\n",
              "      <td>0.760600</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1640</td>\n",
              "      <td>0.763100</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1660</td>\n",
              "      <td>0.756100</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1680</td>\n",
              "      <td>0.702400</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1700</td>\n",
              "      <td>0.752600</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1720</td>\n",
              "      <td>0.787000</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1740</td>\n",
              "      <td>0.702600</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1760</td>\n",
              "      <td>0.758900</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1780</td>\n",
              "      <td>0.751600</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1800</td>\n",
              "      <td>0.734500</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1820</td>\n",
              "      <td>0.739200</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1840</td>\n",
              "      <td>0.720600</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1860</td>\n",
              "      <td>0.704200</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1880</td>\n",
              "      <td>0.725100</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1900</td>\n",
              "      <td>0.723700</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1920</td>\n",
              "      <td>0.688100</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1940</td>\n",
              "      <td>0.690700</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1960</td>\n",
              "      <td>0.732100</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>1980</td>\n",
              "      <td>0.748300</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2000</td>\n",
              "      <td>0.722900</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2020</td>\n",
              "      <td>0.683700</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2040</td>\n",
              "      <td>0.762500</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2060</td>\n",
              "      <td>0.665000</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2080</td>\n",
              "      <td>0.703000</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2100</td>\n",
              "      <td>0.718400</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2120</td>\n",
              "      <td>0.688100</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2140</td>\n",
              "      <td>0.707600</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2160</td>\n",
              "      <td>0.750600</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2180</td>\n",
              "      <td>0.711100</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2200</td>\n",
              "      <td>0.739500</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2220</td>\n",
              "      <td>0.671900</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2240</td>\n",
              "      <td>0.709500</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2260</td>\n",
              "      <td>0.703500</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2280</td>\n",
              "      <td>0.717900</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2300</td>\n",
              "      <td>0.673500</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2320</td>\n",
              "      <td>0.650000</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2340</td>\n",
              "      <td>0.721800</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2360</td>\n",
              "      <td>0.717000</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2380</td>\n",
              "      <td>0.678500</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2400</td>\n",
              "      <td>0.697200</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2420</td>\n",
              "      <td>0.712000</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2440</td>\n",
              "      <td>0.722000</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2460</td>\n",
              "      <td>0.714300</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2480</td>\n",
              "      <td>0.682700</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2500</td>\n",
              "      <td>0.693800</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2520</td>\n",
              "      <td>0.712200</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2540</td>\n",
              "      <td>0.668100</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2560</td>\n",
              "      <td>0.718200</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2580</td>\n",
              "      <td>0.700100</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2600</td>\n",
              "      <td>0.702400</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2620</td>\n",
              "      <td>0.706700</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2640</td>\n",
              "      <td>0.661300</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2660</td>\n",
              "      <td>0.651500</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2680</td>\n",
              "      <td>0.655900</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2700</td>\n",
              "      <td>0.662900</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2720</td>\n",
              "      <td>0.649500</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2740</td>\n",
              "      <td>0.605900</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2760</td>\n",
              "      <td>0.652700</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2780</td>\n",
              "      <td>0.665100</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2800</td>\n",
              "      <td>0.684900</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2820</td>\n",
              "      <td>0.662800</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2840</td>\n",
              "      <td>0.647700</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2860</td>\n",
              "      <td>0.673900</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2880</td>\n",
              "      <td>0.677000</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2900</td>\n",
              "      <td>0.655800</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2920</td>\n",
              "      <td>0.669500</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2940</td>\n",
              "      <td>0.655500</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2960</td>\n",
              "      <td>0.669300</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>2980</td>\n",
              "      <td>0.685000</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3000</td>\n",
              "      <td>0.663300</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3020</td>\n",
              "      <td>0.616500</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3040</td>\n",
              "      <td>0.600700</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3060</td>\n",
              "      <td>0.623500</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3080</td>\n",
              "      <td>0.670300</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3100</td>\n",
              "      <td>0.635500</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3120</td>\n",
              "      <td>0.643600</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3140</td>\n",
              "      <td>0.650500</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3160</td>\n",
              "      <td>0.710800</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3180</td>\n",
              "      <td>0.643600</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3200</td>\n",
              "      <td>0.605300</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3220</td>\n",
              "      <td>0.658400</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3240</td>\n",
              "      <td>0.620800</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3260</td>\n",
              "      <td>0.614900</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3280</td>\n",
              "      <td>0.647400</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3300</td>\n",
              "      <td>0.632700</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3320</td>\n",
              "      <td>0.613100</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3340</td>\n",
              "      <td>0.651600</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3360</td>\n",
              "      <td>0.673200</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3380</td>\n",
              "      <td>0.620400</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3400</td>\n",
              "      <td>0.646400</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3420</td>\n",
              "      <td>0.640300</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3440</td>\n",
              "      <td>0.654400</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3460</td>\n",
              "      <td>0.616300</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3480</td>\n",
              "      <td>0.647800</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3500</td>\n",
              "      <td>0.617500</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3520</td>\n",
              "      <td>0.629100</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3540</td>\n",
              "      <td>0.651400</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3560</td>\n",
              "      <td>0.623400</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3580</td>\n",
              "      <td>0.660200</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3600</td>\n",
              "      <td>0.631900</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3620</td>\n",
              "      <td>0.629000</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3640</td>\n",
              "      <td>0.639000</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3660</td>\n",
              "      <td>0.637500</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3680</td>\n",
              "      <td>0.633800</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3700</td>\n",
              "      <td>0.643000</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3720</td>\n",
              "      <td>0.633800</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3740</td>\n",
              "      <td>0.609000</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3760</td>\n",
              "      <td>0.630800</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3780</td>\n",
              "      <td>0.654000</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3800</td>\n",
              "      <td>0.606300</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3820</td>\n",
              "      <td>0.657900</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3840</td>\n",
              "      <td>0.646800</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3860</td>\n",
              "      <td>0.629100</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3880</td>\n",
              "      <td>0.593100</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3900</td>\n",
              "      <td>0.613600</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3920</td>\n",
              "      <td>0.633300</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3940</td>\n",
              "      <td>0.653200</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3960</td>\n",
              "      <td>0.638600</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>3980</td>\n",
              "      <td>0.612100</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4000</td>\n",
              "      <td>0.644800</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4020</td>\n",
              "      <td>0.610000</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4040</td>\n",
              "      <td>0.583000</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4060</td>\n",
              "      <td>0.618300</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4080</td>\n",
              "      <td>0.641700</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4100</td>\n",
              "      <td>0.600000</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4120</td>\n",
              "      <td>0.622400</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4140</td>\n",
              "      <td>0.622100</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4160</td>\n",
              "      <td>0.674700</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4180</td>\n",
              "      <td>0.610600</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4200</td>\n",
              "      <td>0.652700</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4220</td>\n",
              "      <td>0.657400</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4240</td>\n",
              "      <td>0.665100</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4260</td>\n",
              "      <td>0.658900</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4280</td>\n",
              "      <td>0.623000</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4300</td>\n",
              "      <td>0.695100</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4320</td>\n",
              "      <td>0.630700</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4340</td>\n",
              "      <td>0.613600</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4360</td>\n",
              "      <td>0.619700</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4380</td>\n",
              "      <td>0.618500</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4400</td>\n",
              "      <td>0.617100</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4420</td>\n",
              "      <td>0.615100</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4440</td>\n",
              "      <td>0.645500</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4460</td>\n",
              "      <td>0.628800</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4480</td>\n",
              "      <td>0.596600</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4500</td>\n",
              "      <td>0.623500</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4520</td>\n",
              "      <td>0.603300</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4540</td>\n",
              "      <td>0.650500</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4560</td>\n",
              "      <td>0.623800</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4580</td>\n",
              "      <td>0.607700</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4600</td>\n",
              "      <td>0.647100</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4620</td>\n",
              "      <td>0.602600</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4640</td>\n",
              "      <td>0.636100</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4660</td>\n",
              "      <td>0.635800</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4680</td>\n",
              "      <td>0.619700</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4700</td>\n",
              "      <td>0.626800</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4720</td>\n",
              "      <td>0.649600</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4740</td>\n",
              "      <td>0.598600</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4760</td>\n",
              "      <td>0.666200</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4780</td>\n",
              "      <td>0.621400</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4800</td>\n",
              "      <td>0.629800</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4820</td>\n",
              "      <td>0.613400</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4840</td>\n",
              "      <td>0.629600</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4860</td>\n",
              "      <td>0.604000</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4880</td>\n",
              "      <td>0.621900</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4900</td>\n",
              "      <td>0.590700</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4920</td>\n",
              "      <td>0.585000</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4940</td>\n",
              "      <td>0.593100</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4960</td>\n",
              "      <td>0.589300</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>4980</td>\n",
              "      <td>0.597900</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>5000</td>\n",
              "      <td>0.621500</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>5020</td>\n",
              "      <td>0.639900</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>5040</td>\n",
              "      <td>0.589100</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>5060</td>\n",
              "      <td>0.633700</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>5080</td>\n",
              "      <td>0.656900</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>5100</td>\n",
              "      <td>0.646900</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>5120</td>\n",
              "      <td>0.603500</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>5140</td>\n",
              "      <td>0.644700</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>5160</td>\n",
              "      <td>0.625300</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>5180</td>\n",
              "      <td>0.632300</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>5200</td>\n",
              "      <td>0.621500</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>5220</td>\n",
              "      <td>0.588000</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>5240</td>\n",
              "      <td>0.616200</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>5260</td>\n",
              "      <td>0.594300</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>5280</td>\n",
              "      <td>0.647000</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>5300</td>\n",
              "      <td>0.648700</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>5320</td>\n",
              "      <td>0.601900</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>5340</td>\n",
              "      <td>0.618700</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <td>5360</td>\n",
              "      <td>0.627800</td>\n",
              "    </tr>\n",
              "  </tbody>\n",
              "</table><p>"
            ]
          },
          "metadata": {}
        },
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "\n",
            "Training completed in 63 minutes and 8 seconds.\n"
          ]
        }
      ],
      "source": [
        "# Step 9: Start training and measure time\n",
        "print(\"\\nStarting training...\")\n",
        "start_time = time.time()  # Start timer\n",
        "\n",
        "trainer.train()  # Begin model training\n",
        "\n",
        "end_time = time.time()  # End timer\n",
        "duration = end_time - start_time\n",
        "minutes, seconds = divmod(duration, 60)\n",
        "print(f\"\\nTraining completed in {int(minutes)} minutes and {int(seconds)} seconds.\")\n"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 11,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "1RXE1-TaO0T9",
        "outputId": "6536bb81-b8f4-496e-ebc5-9a082c4703d3"
      },
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Memory usage after training: 8.92 GB\n"
          ]
        }
      ],
      "source": [
        "# Step 10: Check memory after training\n",
        "print(f\"Memory usage after training: {psutil.virtual_memory().available / (1024 ** 3):.2f} GB\")\n"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 12,
      "metadata": {
        "id": "eIHENDSiO3pC",
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 106
        },
        "outputId": "dc7c37ae-5f05-4f65-ed4d-16b567e92388"
      },
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "\n",
            "🔍 Evaluating the model on GPU...\n"
          ]
        },
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<IPython.core.display.HTML object>"
            ],
            "text/html": [
              "\n",
              "    <div>\n",
              "      \n",
              "      <progress value='672' max='672' style='width:300px; height:20px; vertical-align: middle;'></progress>\n",
              "      [672/672 01:58]\n",
              "    </div>\n",
              "    "
            ]
          },
          "metadata": {}
        },
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "\n",
            "✅ Model and tokenizer saved to 'voxa_model_cpu' directory.\n"
          ]
        }
      ],
      "source": [
        "#  STEP 11: Evaluate model performance and save everything\n",
        "\n",
        "print(\"\\n🔍 Evaluating the model on GPU...\")\n",
        "eval_results = trainer.evaluate()\n",
        "\n",
        "# Save the model and tokenizer so we can load them later\n",
        "model.save_pretrained(\"voxa_model_cpu\")\n",
        "tokenizer.save_pretrained(\"voxa_model_cpu\")\n",
        "print(\"\\n✅ Model and tokenizer saved to 'voxa_model_cpu' directory.\")\n",
        "\n"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 14,
      "metadata": {
        "id": "x7gA_SNNO88G",
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "outputId": "de1dce39-cea1-42e6-9ca8-90f9148af0e4"
      },
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Mounted at /content/drive\n",
            "\n",
            "📦 Creating ZIP archive of the model directory...\n",
            "✅ Archive created at: /content/voxa_model_cpu.zip\n",
            "\n",
            "📁 Saving ZIP to Google Drive...\n",
            "✅ ZIP file saved at: /content/drive/MyDrive/voxa_model_cpu.zip\n"
          ]
        }
      ],
      "source": [
        "# step 12 : save model and download from google drive\n",
        "import shutil\n",
        "from google.colab import drive\n",
        "\n",
        "# Mount Google Drive\n",
        "drive.mount('/content/drive')\n",
        "\n",
        "# Path where the model will be saved inside your Google Drive\n",
        "drive_save_path = \"/content/drive/MyDrive/voxa_model_cpu.zip\"\n",
        "\n",
        "# Compress the 'voxa_model_cpu' folder into a zip archive\n",
        "print(\"\\n📦 Creating ZIP archive of the model directory...\")\n",
        "shutil.make_archive(\"/content/voxa_model_cpu\", 'zip', 'voxa_model_cpu')\n",
        "print(\"✅ Archive created at: /content/voxa_model_cpu.zip\")\n",
        "\n",
        "# Copy the ZIP file to Google Drive\n",
        "print(\"\\n📁 Saving ZIP to Google Drive...\")\n",
        "shutil.copy(\"/content/voxa_model_cpu.zip\", drive_save_path)\n",
        "print(f\"✅ ZIP file saved at: {drive_save_path}\")\n"
      ]
    },
    {
      "cell_type": "code",
      "source": [],
      "metadata": {
        "id": "JfaWOHCXqbHB"
      },
      "execution_count": null,
      "outputs": []
    }
  ],
  "metadata": {
    "accelerator": "GPU",
    "colab": {
      "gpuType": "T4",
      "provenance": []
    },
    "kernelspec": {
      "display_name": "Python 3",
      "name": "python3"
    },
    "language_info": {
      "name": "python"
    },
    "widgets": {
      "application/vnd.jupyter.widget-state+json": {
        "c777f1d7b051452dacd3a15722064ac7": {
          "model_module": "@jupyter-widgets/controls",
          "model_name": "HBoxModel",
          "model_module_version": "1.5.0",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_aa6463e797284971b88ac46c38dc46c1",
              "IPY_MODEL_ac4949791ae3483c87960b9af72164b4",
              "IPY_MODEL_ca351d8c794740df8a627b8774f98f6d"
            ],
            "layout": "IPY_MODEL_8ab3b9b7030d4882ab5c8563e9a8951b"
          }
        },
        "aa6463e797284971b88ac46c38dc46c1": {
          "model_module": "@jupyter-widgets/controls",
          "model_name": "HTMLModel",
          "model_module_version": "1.5.0",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_42973b38546c421aae760a0db8a848d5",
            "placeholder": "​",
            "style": "IPY_MODEL_6c79457fd1ee4e20b83b5d7058d17a37",
            "value": "Map: 100%"
          }
        },
        "ac4949791ae3483c87960b9af72164b4": {
          "model_module": "@jupyter-widgets/controls",
          "model_name": "FloatProgressModel",
          "model_module_version": "1.5.0",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_d4acf8efe5834f9abed1b63bd983c97a",
            "max": 21497,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_01ecbe20bf6648588cbddb378632a7b7",
            "value": 21497
          }
        },
        "ca351d8c794740df8a627b8774f98f6d": {
          "model_module": "@jupyter-widgets/controls",
          "model_name": "HTMLModel",
          "model_module_version": "1.5.0",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_102c219e9e0044618895f0cc27a01a80",
            "placeholder": "​",
            "style": "IPY_MODEL_00e597996aac4637a8cbd3057c04d19a",
            "value": " 21497/21497 [00:22&lt;00:00, 1180.53 examples/s]"
          }
        },
        "8ab3b9b7030d4882ab5c8563e9a8951b": {
          "model_module": "@jupyter-widgets/base",
          "model_name": "LayoutModel",
          "model_module_version": "1.2.0",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "42973b38546c421aae760a0db8a848d5": {
          "model_module": "@jupyter-widgets/base",
          "model_name": "LayoutModel",
          "model_module_version": "1.2.0",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "6c79457fd1ee4e20b83b5d7058d17a37": {
          "model_module": "@jupyter-widgets/controls",
          "model_name": "DescriptionStyleModel",
          "model_module_version": "1.5.0",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "d4acf8efe5834f9abed1b63bd983c97a": {
          "model_module": "@jupyter-widgets/base",
          "model_name": "LayoutModel",
          "model_module_version": "1.2.0",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "01ecbe20bf6648588cbddb378632a7b7": {
          "model_module": "@jupyter-widgets/controls",
          "model_name": "ProgressStyleModel",
          "model_module_version": "1.5.0",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "102c219e9e0044618895f0cc27a01a80": {
          "model_module": "@jupyter-widgets/base",
          "model_name": "LayoutModel",
          "model_module_version": "1.2.0",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "00e597996aac4637a8cbd3057c04d19a": {
          "model_module": "@jupyter-widgets/controls",
          "model_name": "DescriptionStyleModel",
          "model_module_version": "1.5.0",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "0b5fa0aa83bb43d79b424c10464d8151": {
          "model_module": "@jupyter-widgets/controls",
          "model_name": "HBoxModel",
          "model_module_version": "1.5.0",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HBoxModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HBoxView",
            "box_style": "",
            "children": [
              "IPY_MODEL_843762fb47b84b408106c9e3a55a156c",
              "IPY_MODEL_5095d04062af413cb4fbc4d66009056e",
              "IPY_MODEL_c0359cf4432f4b5e9b5eef4d7aab21ea"
            ],
            "layout": "IPY_MODEL_70b7a6fd920a4e50bca9b29fb0f0dc9c"
          }
        },
        "843762fb47b84b408106c9e3a55a156c": {
          "model_module": "@jupyter-widgets/controls",
          "model_name": "HTMLModel",
          "model_module_version": "1.5.0",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_3f9959767bd8400fac30d7a53e787a67",
            "placeholder": "​",
            "style": "IPY_MODEL_403a3fdb37e24846a503a992e5eeea54",
            "value": "Map: 100%"
          }
        },
        "5095d04062af413cb4fbc4d66009056e": {
          "model_module": "@jupyter-widgets/controls",
          "model_name": "FloatProgressModel",
          "model_module_version": "1.5.0",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "FloatProgressModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "ProgressView",
            "bar_style": "success",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_4dd33e1517684c92a1da2fb22a2a5368",
            "max": 5375,
            "min": 0,
            "orientation": "horizontal",
            "style": "IPY_MODEL_b78dfd88cb01423abe75df88048f77c4",
            "value": 5375
          }
        },
        "c0359cf4432f4b5e9b5eef4d7aab21ea": {
          "model_module": "@jupyter-widgets/controls",
          "model_name": "HTMLModel",
          "model_module_version": "1.5.0",
          "state": {
            "_dom_classes": [],
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "HTMLModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/controls",
            "_view_module_version": "1.5.0",
            "_view_name": "HTMLView",
            "description": "",
            "description_tooltip": null,
            "layout": "IPY_MODEL_670e28c91f8f49b58acf64ad10a9db51",
            "placeholder": "​",
            "style": "IPY_MODEL_561b83b423cc438f86ba3ca5f17c3aaf",
            "value": " 5375/5375 [00:05&lt;00:00, 791.81 examples/s]"
          }
        },
        "70b7a6fd920a4e50bca9b29fb0f0dc9c": {
          "model_module": "@jupyter-widgets/base",
          "model_name": "LayoutModel",
          "model_module_version": "1.2.0",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "3f9959767bd8400fac30d7a53e787a67": {
          "model_module": "@jupyter-widgets/base",
          "model_name": "LayoutModel",
          "model_module_version": "1.2.0",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "403a3fdb37e24846a503a992e5eeea54": {
          "model_module": "@jupyter-widgets/controls",
          "model_name": "DescriptionStyleModel",
          "model_module_version": "1.5.0",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        },
        "4dd33e1517684c92a1da2fb22a2a5368": {
          "model_module": "@jupyter-widgets/base",
          "model_name": "LayoutModel",
          "model_module_version": "1.2.0",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "b78dfd88cb01423abe75df88048f77c4": {
          "model_module": "@jupyter-widgets/controls",
          "model_name": "ProgressStyleModel",
          "model_module_version": "1.5.0",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "ProgressStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "bar_color": null,
            "description_width": ""
          }
        },
        "670e28c91f8f49b58acf64ad10a9db51": {
          "model_module": "@jupyter-widgets/base",
          "model_name": "LayoutModel",
          "model_module_version": "1.2.0",
          "state": {
            "_model_module": "@jupyter-widgets/base",
            "_model_module_version": "1.2.0",
            "_model_name": "LayoutModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "LayoutView",
            "align_content": null,
            "align_items": null,
            "align_self": null,
            "border": null,
            "bottom": null,
            "display": null,
            "flex": null,
            "flex_flow": null,
            "grid_area": null,
            "grid_auto_columns": null,
            "grid_auto_flow": null,
            "grid_auto_rows": null,
            "grid_column": null,
            "grid_gap": null,
            "grid_row": null,
            "grid_template_areas": null,
            "grid_template_columns": null,
            "grid_template_rows": null,
            "height": null,
            "justify_content": null,
            "justify_items": null,
            "left": null,
            "margin": null,
            "max_height": null,
            "max_width": null,
            "min_height": null,
            "min_width": null,
            "object_fit": null,
            "object_position": null,
            "order": null,
            "overflow": null,
            "overflow_x": null,
            "overflow_y": null,
            "padding": null,
            "right": null,
            "top": null,
            "visibility": null,
            "width": null
          }
        },
        "561b83b423cc438f86ba3ca5f17c3aaf": {
          "model_module": "@jupyter-widgets/controls",
          "model_name": "DescriptionStyleModel",
          "model_module_version": "1.5.0",
          "state": {
            "_model_module": "@jupyter-widgets/controls",
            "_model_module_version": "1.5.0",
            "_model_name": "DescriptionStyleModel",
            "_view_count": null,
            "_view_module": "@jupyter-widgets/base",
            "_view_module_version": "1.2.0",
            "_view_name": "StyleView",
            "description_width": ""
          }
        }
      }
    }
  },
  "nbformat": 4,
  "nbformat_minor": 0
}